<a href="https://colab.research.google.com/github/dtabuena/EphysLib/blob/main/_Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
'Get Standard Modules'

import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import scipy
from scipy import stats
import os
from scipy.signal import butter,filtfilt
from tqdm import tqdm
import warnings
from IPython.display import clear_output
from datetime import datetime
import sys
import warnings
import shutil
from google.colab import files
warnings.filterwarnings('ignore')
np.set_printoptions(threshold=sys.maxsize)
clear_output(wait=False)

In [10]:
try: shutil.rmtree('/content/EphysLib')
except: None

"run dtabuena's ephys notebooks"
!git clone https://github.com/dtabuena/EphysLib

to_import = ['ABF_Quality_Control.ipynb',
          'Basic_Ephys.ipynb',
          'Firing_Rate_Gain.ipynb',
          'Simple_ABF_tools.ipynb',
          'IV_analyzer.ipynb',
          'membrane_analyzer.ipynb',
          'analyze_rheobase.ipynb',
          'fun_math.ipynb',
          'importing_abfs_from_dropbox.ipynb',
          'input_resistance_analyzer.ipynb',
          'latencey_analyzer.ipynb',
          'QC_recoding_dataframe.ipynb'
            ]

for i in to_import:
    f = '/content/EphysLib/' + i
    %run $f


Cloning into 'EphysLib'...
remote: Enumerating objects: 181, done.
remote: Counting objects: 100% (87/87), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 181 (delta 58), reused 29 (delta 29), pack-reused 94
Receiving objects: 100% (181/181), 180.10 KiB | 20.01 MiB/s, done.
Resolving deltas: 100% (102/102), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [11]:
'Import and catalog source data'
'RNF182 Dropbox'
data_source = "https://www.dropbox.com/sh/n9t8p257wnzlijk/AAC9Z36JodisyZjnM3mkJC3Xa?dl=0"
file_loc = get_drobox_folder(data_source, 'my_ephys_data.zip')
clear_output(wait=False)
file_naming_scheme = ['Rec_date','Virus','GenoType','Sex','Age','Slice_Num','Cell_num','Cell_Type']
abf_recordings_df,protocol_set = catalogue_recs(file_loc,file_naming_scheme)
print('Protocol_Names:')
_ = [print(p) for p in protocol_set]

Protocol_Names:
VC - Multi IV - 150ms
IC - Rheobase
IC - Gain - D50pA
IC - Sag - D50pA
IC - R input
VC - MemTest-10ms-160ms
VC - 3min GapFree
IC - Latentcy 800pA-1s


In [12]:
VC_prot = ['VC - MemTest-10ms-160ms',
           'VC - Multi IV - 150ms',
           'VC - 3min GapFree',
           'VC - Spontaneous-3min-(MT)']
IC_prot = ['IC - Gain - D20pA',
           'IC - Gain - D50pA',
           'IC - Rheobase',
           'IC - R input',
           'IC - Latentcy 800pA-1s']

abf_recordings_df, _ = purge_wrong_clamp(abf_recordings_df,VC_prot,IC_prot)

In [13]:
def analysis_iterator(abf_recordings_df,func_dict,arg_dict):
    problem_recs = []
    def init_col_object(df,name): 
        df[name] = None
        df[name] = df[name].astype(object)
        return df

    for file_name in tqdm(abf_recordings_df.index):
        abf = abf_or_name(file_name)
        prot_name = abf.protocol


        # check for keyed protocol
        if prot_name not in func_dict.keys():
            # print('unknown protocol(func): ',  prot_name)
            continue
        if prot_name not in arg_dict.keys():
            # print('unknown protocol(args): ',  prot_name)
            continue


        try:
            analyzer_func = func_dict[prot_name]  # get analyzer from dict
            args_for_analyzer =  [abf] + arg_dict[prot_name] # get args for analyzer from dict
            results = analyzer_func(*args_for_analyzer) # run analyzer
            for k in results.keys():

                # New Col?
                cols = abf_recordings_df.columns
                if k not in cols:
                    abf_recordings_df = init_col_object(abf_recordings_df,k)
                abf_recordings_df.at[file_name,k] = results[k]
        except: 
            print('error on: ' ,file_name)
            problem_recs.append(file_name)

    return abf_recordings_df, problem_recs


spike_args =  {'spike_thresh':20, 'high_dv_thresh': 25,'low_dv_thresh': -5,'window_ms': 2}

func_dict = {}
arg_dict = {}

func_dict['IC - Rheobase']= rheobase_analyzer
arg_dict['IC - Rheobase'] = [spike_args, False, False, False]  # [spike_args, to_plot, verbose, force_singlespike]

func_dict['IC - Gain - D20pA']= gain_analyzer
arg_dict['IC - Gain - D20pA']= [spike_args, 0.8, 0]  # [spike_args, to_plot, verbose, force_singlespike]
func_dict['IC - Gain - D50pA']= func_dict['IC - Gain - D20pA'] 
arg_dict['IC - Gain - D50pA']= arg_dict['IC - Gain - D20pA']

func_dict['VC - MemTest-10ms-160ms']= membrane_analyzer
arg_dict['VC - MemTest-10ms-160ms']= [False, False, ['Ra', 'Rm', 'Cm', 'tau',	'Cmq',	'Cmf',	'Cmqf', 'Cm_pc']]  # [to_plot, verbose]

func_dict['IC - Latentcy 800pA-1s']= latencey_analyzer 
arg_dict['IC - Latentcy 800pA-1s']= [spike_args, False]  # [spike_args, to_plot]

func_dict['IC - Latentcy 800pA-1s']= latencey_analyzer 
arg_dict['IC - Latentcy 800pA-1s']= [spike_args, False]  # [spike_args, to_plot]

func_dict['IC - R input']= input_resistance_analyzer 
arg_dict['IC - R input']= [[-30, 10] ,False]  # [dVm_limits, to_plot]


func_dict['VC - Multi IV - 150ms'] = IV_analyzer
arg_dict['VC - Multi IV - 150ms']= [{'IV_Early':(16.5, 30),'IV_Steady_State':(100,120)} ,False]  # [measure_windows, to_plot]


abf_recordings_df, problem_recs = analysis_iterator(abf_recordings_df,func_dict,arg_dict)
print(problem_recs)

abf_recordings_df.to_csv('test.csv')
files.download('test.csv')

 25%|██▌       | 57/226 [00:22<00:37,  4.52it/s]

error on:  my_ephys_data/2022_08_12_RNF182/2022x08x12_RNF182_E4KI_F_P251_s002_c006_CA3xPOS_0006.abf


 35%|███▌      | 80/226 [00:31<00:47,  3.11it/s]

ap_stats_failed:  ABF (v2.9) with 2 channels (mV, pA), sampled at 10.0 kHz, containing 4 sweeps, having no tags, with a total length of 0.73 minutes, recorded with protocol "IC - Rheobase".


100%|██████████| 226/226 [01:30<00:00,  2.48it/s]

['my_ephys_data/2022_08_12_RNF182/2022x08x12_RNF182_E4KI_F_P251_s002_c006_CA3xPOS_0006.abf']


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [14]:
def cell_sorting(abf_recordings_df):

    unique_cells = list(set(abf_recordings_df['cell_id']))
    unique_cells.sort()
    transfer_cols = [c for c in abf_recordings_df.columns if 'cell_id' not in c]
    cell_df = pd.DataFrame(index=list(unique_cells),columns = transfer_cols)
    

    for cell in cell_df.index:
        match = [cell in r for r in abf_recordings_df['cell_id']]
        for col in transfer_cols:
            match_values = list(abf_recordings_df[match][col])
            # print('col', col)
            # print(match_values)

            cell_df.at[cell,col] = match_values
    return cell_df


cell_df = cell_sorting(abf_recordings_df)

def cell_consolidation(cell_df,list_types,any_types,average_types = True):
    cell_df_con = cell_df.copy()
    explicit_cols = ['IV_Early','IV_Steady_State']

    if average_types:
        average_types = [c for c in cell_df_con.columns if c not in any_types and c not in list_types and c not in explicit_cols]
        
        # print('average_types',average_types)



    for cell in cell_df_con.index:
        for col in list_types:
            'do nothing, keep the list'
        for col in any_types:
            'they are all the same take the first'
            cell_df_con.at[cell,col] = cell_df_con.at[cell,col][0]

        for col in average_types:
            multi_vals = cell_df_con.loc[cell,col]
            try:
                multi_vals = [v for v in multi_vals if v is not None]
                single_val = np.nanmean(multi_vals,0)
                cell_df_con.at[cell,col] = single_val
                # print(single_val)
            except: 'Just keep going None'
        

    # explicitly defined consolidation
    for cell in cell_df_con.index:
        try:
            multi_vals = cell_df_con.loc[cell,'IV_Early']
            multi_vals = [v for v in multi_vals if v is not None]            
            v_stim = [  mv['V_stim'] for mv in  multi_vals ]
            peak_vals = [  mv['I_peak'] for mv in  multi_vals ]
            if len(v_stim)>1:
                v_stim_set = [v for v in [vl for vl in  v_stim ] ]
                print(v_stim_set)

        except:
            if np.isnan(multi_vals):
                multi_vals = None
            

        # print(multi_vals)

    return cell_df_con

list_types = ['Recording_name','protocol','abf_timestamp', 'channelList']
any_types = ['Rec_date',	'Virus',	'GenoType',	'Sex',	'Age',	'Slice_Num',	'Cell_num',	'Cell_Type']


cell_df_con = cell_consolidation(cell_df,list_types,any_types)
cell_df_con.to_csv('cell_df_con.csv')
files.download('cell_df_con.csv')

[[-130], [-130, -120, -110, -100, -90, -80, -70, -60, -50, -40, -30, -20, -10, 0, 10]]


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>